In [ ]:
input_shape = (128, 128, 3)
num_conv_filters = 64  # Example filter size
filter_growth_rate = 2  # Example growth rate
dropout_rate = 0.5  # Dropout rate

def decoder_block(input_tensor, skip_connection, num_filters, filter_size=(3, 3), dropout_rate=0.0):
    """Decoder block for the U-Net architecture.

    Args:
        input_tensor: Input tensor
        skip_connection: Skip connection tensor
        num_filters: Number of filters for the convolutional layers
        filter_size: Size of the convolutional filters
        dropout_rate: Dropout rate
    """

    # Upsample the input tensor
    x = UpSampling2D((2, 2))(input_tensor)

    # Concatenate the upsampled input tensor with the skip connection
    x = Concatenate()([x, skip_connection])

    # Convolutional layer
    x = Conv2D(num_filters, filter_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Convolutional layer
    x = Conv2D(num_filters, filter_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Apply dropout if needed
    if dropout_rate > 0:
        x = Dropout(dropout_rate)(x)

    return x

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, UpSampling2D, Dropout, ReLU, Input, Concatenate
from tensorflow.keras.applications import ResNet50

def build_resnet50_unet(input_shape, num_conv_filters=64, filter_growth_rate=2, dropout_rate=0.3):
    # Input layer
    inputs = Input(shape=input_shape)

    # Load the ResNet50 model without the fully connected layers and with pretrained ImageNet weights
    # Ensure the input tensor is passed to the model here
    resnet_encoder = ResNet50(input_tensor=inputs, include_top=False, weights='imagenet')

    # Extract feature maps from specific layers of ResNet50 for skip connections
    skip1 = resnet_encoder.get_layer('conv1_relu').output  # Shape: (64, 64, 64)
    skip2 = resnet_encoder.get_layer('conv2_block3_out').output  # Shape: (32, 32, 256)
    skip3 = resnet_encoder.get_layer('conv3_block4_out').output  # Shape: (16, 16, 512)
    skip4 = resnet_encoder.get_layer('conv4_block6_out').output  # Shape: (8, 8, 1024)

    # Bottleneck (deepest layer from ResNet50)
    bottleneck = resnet_encoder.get_layer('conv5_block3_out').output  # Shape: (4, 4, 2048)

    # Bottleneck Conv layers
    c3 = Conv2D(num_conv_filters * filter_growth_rate * 2, (3, 3), padding='same')(bottleneck)
    c3 = ReLU()(c3)
    c3 = BatchNormalization()(c3)
    c3 = Conv2D(num_conv_filters * filter_growth_rate * 2, (3, 3), padding='same')(c3)
    c3 = ReLU()(c3)
    c3 = BatchNormalization()(c3)

    # Decoder Block 1: Upsample from (4,4) -> (8,8), skip connection from skip4
    u4 = decoder_block(c3, skip4, num_conv_filters * filter_growth_rate * 2, dropout_rate=dropout_rate)

    # Decoder Block 2: Upsample from (8,8) -> (16,16), skip connection from skip3
    u5 = decoder_block(u4, skip3, num_conv_filters * filter_growth_rate * 2, dropout_rate=dropout_rate)

    # Decoder Block 3: Upsample from (16,16) -> (32,32), skip connection from skip2
    u6 = decoder_block(u5, skip2, num_conv_filters * filter_growth_rate, dropout_rate=dropout_rate)

    # Decoder Block 4: Upsample from (32,32) -> (64,64), skip connection from skip1
    u7 = decoder_block(u6, skip1, num_conv_filters, dropout_rate=dropout_rate)

    # Decoder Block 5: Upsample from (64,64) -> (128,128) without a skip connection
    u8 = UpSampling2D((2, 2))(u7)
    u8 = Conv2D(num_conv_filters, (3, 3), padding='same')(u8)
    u8 = ReLU()(u8)
    u8 = BatchNormalization()(u8)

    # Apply Dropout if needed
    if dropout_rate > 0:
        u8 = Dropout(dropout_rate)(u8)

    # Final output layer (1-channel output for binary segmentation)
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(u8)

    # Build the final model
    model = Model(inputs, outputs)

    return model

# Build the model
model = build_resnet50_unet(input_shape, num_conv_filters=32, filter_growth_rate=4, dropout_rate=0.5)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, UpSampling2D, Dropout, ReLU, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.metrics import TruePositives, TrueNegatives, FalsePositives, FalseNegatives

def build_efficientnetv2_unet(input_shape, num_conv_filters=64, filter_growth_rate=2, dropout_rate=0.3):
    # Input layer
    inputs = Input(shape=input_shape)

    # Load the EfficientNetV2 model without the fully connected layers and with pretrained ImageNet weights
    effnet_encoder = EfficientNetV2B0(input_tensor=inputs, include_top=False, weights='imagenet')

    # Extract feature maps from specific layers of EfficientNetV2 for skip connections
    skip1 = effnet_encoder.get_layer('block2a_expand_activation').output  # Shape: (64, 64, 32)
    skip2 = effnet_encoder.get_layer('block3a_expand_activation').output  # Shape: (32, 32, 48)
    skip3 = effnet_encoder.get_layer('block4a_expand_activation').output  # Shape: (16, 16, 112)
    skip4 = effnet_encoder.get_layer('block6a_expand_activation').output  # Shape: (8, 8, 192)

    # Bottleneck (deepest layer from EfficientNetV2)
    bottleneck = effnet_encoder.get_layer('top_activation').output  # Shape: (4, 4, 1280)

    # Bottleneck Conv layers
    c3 = Conv2D(num_conv_filters * filter_growth_rate * 2, (3, 3), padding='same')(bottleneck)
    c3 = ReLU()(c3)
    c3 = BatchNormalization()(c3)
    c3 = Conv2D(num_conv_filters * filter_growth_rate * 2, (3, 3), padding='same')(c3)
    c3 = ReLU()(c3)
    c3 = BatchNormalization()(c3)

    # Decoder Block 1: Upsample from (4,4) -> (8,8), skip connection from skip4
    u4 = UpSampling2D((2, 2))(c3)
    u4 = Concatenate()([u4, skip4])  # Add skip connection
    u4 = Conv2D(num_conv_filters * filter_growth_rate * 2, (3, 3), padding='same')(u4)
    u4 = ReLU()(u4)
    u4 = BatchNormalization()(u4)

    # Decoder Block 2: Upsample from (8,8) -> (16,16), skip connection from skip3
    u5 = UpSampling2D((2, 2))(u4)
    u5 = Concatenate()([u5, skip3])  # Add skip connection to (16,16,112)
    u5 = Conv2D(num_conv_filters * filter_growth_rate * 2, (3, 3), padding='same')(u5)
    u5 = ReLU()(u5)
    u5 = BatchNormalization()(u5)

    # Decoder Block 3: Upsample from (16,16) -> (32,32), skip connection from skip2
    u6 = Concatenate()([u5, skip2])  # Add skip connection
    u6 = UpSampling2D((2, 2))(u6)
    # skip2_upsampled = UpSampling2D((2, 2))(skip2)  # Additional upsampling to match shape
    u6 = Conv2D(num_conv_filters * filter_growth_rate, (3, 3), padding='same')(u6)
    u6 = ReLU()(u6)
    u6 = BatchNormalization()(u6)

    # Decoder Block 4: Upsample from (32,32) -> (64,64), skip connection from skip1
    u7 = Concatenate()([u6, skip1])  # Add skip connection
    u7 = UpSampling2D((2, 2))(u6)
    # skip1_upsampled = UpSampling2D((2, 2))(skip1)  # Additional upsampling to match shape
    u7 = Conv2D(num_conv_filters, (3, 3), padding='same')(u7)
    u7 = ReLU()(u7)
    u7 = BatchNormalization()(u7)

    # Decoder Block 5: Upsample from (64,64) -> (128,128)
    u8 = UpSampling2D((2, 2))(u7)
    u8 = Conv2D(num_conv_filters, (3, 3), padding='same')(u8)
    u8 = ReLU()(u8)
    u8 = BatchNormalization()(u8)

    # Apply Dropout if needed
    if dropout_rate > 0:
        u8 = Dropout(dropout_rate)(u8)

    # Final output layer (1-channel output for binary segmentation)
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(u8)

    # Build the final model
    model = Model(inputs, outputs)

    return model

# Build the model
input_shape = (128, 128, 3)
model = build_efficientnetv2_unet(input_shape, num_conv_filters=16, filter_growth_rate=2, dropout_rate=0.5)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', F1_score, TruePositives(), TrueNegatives(), FalsePositives(), FalseNegatives()])
